In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import adjustText
import pathlib
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
%matplotlib inline
import matplotlib.pyplot as plt
from plotnine import *
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
GIT_PYTHON_REFRESH=0

C:\Users\light\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


## Functions

In [3]:
def make_data_imbens(df):
    df_new=df.drop(['nodegree'], axis=1)
    df_new['pos74']=(df_new['RE74']>0).astype(int)
    df_new['pos75']=(df_new['RE75']>0).astype(int)
    return df_new
    

In [4]:
#made better forest

In [5]:
def predict_g(X, y, do_SMOTE, logs=False):
    np.random.seed(42)
    predictions=np.full_like(y, np.nan, dtype=float)      
    kf=StratifiedKFold(n_splits=10, shuffle=True, random_state=42)   
    for train_index, test_index in kf.split(X,y):
        clf=RandomForestClassifier(random_state=42, oob_score=False, n_estimators=N_EST, max_depth=MAX_DEPTH)
        if do_SMOTE:
            X_balanced,y_balanced=SMOTE().fit_resample(X[train_index], y[train_index])
            clf.fit(X_balanced, y_balanced)
        else:
            clf.fit(X[train_index], y[train_index])
        predictions[test_index]=clf.predict_proba(X[test_index])[:,1]
    assert np.isnan(predictions).sum()==0
    if logs:
        mlflow.log_param('n_est', N_EST)
        mlflow.log_param('max_depth', MAX_DEPTH)
        mlflow.log_param('g_SMOTE', do_SMOTE)
        mlflow.log_metric('g_accuracy', accuracy_score(np.rint(predictions), y))
        column_names=['age', 'education', 'black', 'hispanic', 'married', 'RE74', 'RE75', 'pos74', 'pos75']
        feature_importances=pd.DataFrame({'variables': column_names, 'score': clf.feature_importances_}).sort_values(by='score', ascending=False)
        feature_importances.to_csv('temp_mlflow_files/g_feat_imp.csv')
        mlflow.log_artifact('temp_mlflow_files/g_feat_imp.csv')
    return predictions
    

In [6]:
#made better forest

In [28]:
def predict_Q(X, y, logs=False):
    np.random.seed(42)
    predictions=np.full_like(y, np.nan, dtype=float)
    kf=KFold(n_splits=10, shuffle=True, random_state=42)
    for train_index, test_index in kf.split(X):
        reg=RandomForestRegressor(random_state=42, n_estimators=N_EST, max_depth=MAX_DEPTH)
        reg.fit(X[train_index], y[train_index])
        predictions[test_index]=reg.predict(X[test_index])
    assert np.isnan(predictions).sum()==0
    if logs:
        mlflow.log_metric('q_rmse', np.sqrt(mean_squared_error(predictions, y)))
        column_names=['treatment', 'age', 'education', 'black', 'hispanic', 'married', 'RE74', 'RE75', 'pos74', 'pos75']
        feature_importances=pd.DataFrame({'variables': column_names, 'score': reg.feature_importances_}).sort_values(by='score', ascending=False)
        feature_importances.to_csv('temp_mlflow_files/q_feat_imp.csv')
        mlflow.log_artifact('temp_mlflow_files/q_feat_imp.csv')
    return predictions

In [8]:
def calc_delta(alpha, g, bias):
    alpha_term=(1/alpha)-1
    bias_term=sp.special.digamma(g*alpha_term+1)\
        -sp.special.digamma((1-g)*alpha_term)\
        -sp.special.digamma(g*alpha_term)\
        +sp.special.digamma((1-g)*alpha_term+1)
    delta=bias/np.mean(bias_term)
    return delta

In [9]:
def calc_beta_shapes(g, alpha):
    alpha_shape=g*((1/alpha)-1)
    beta_shape=(1-g)*((1/alpha)-1)
    return alpha_shape, beta_shape
    

def calc_Rsq(alpha, delta, calculated_params, treatment, response):
    alpha_shape, beta_shape=calc_beta_shapes(calculated_params['g'], alpha)
    Rsq_num=delta**2*np.mean(sp.special.polygamma(1, alpha_shape+treatment)\
                             +sp.special.polygamma(1, beta_shape+(1-treatment)))
    Rsq_den=mean_squared_error(response, calculated_params['Q'])
    return Rsq_num/Rsq_den
    
    

In [10]:
def calc_Rsq_hat(covariate_col, response_col, df, Q):
    y=df[response_col].values
    Qhat=predict_Q(df.drop([response_col]+covariate_col, axis=1).values, y)
    Rsqhat=(mean_squared_error(y, Qhat)-mean_squared_error(y, Q))/(mean_squared_error(y, Qhat))
    if Rsqhat<0:
        Rsqhat=0
    return Rsqhat

In [11]:
def calc_alpha_hat(covariate_col, response_col, treatment_col, df, g, do_SMOTE):
    #call y t
    t=df[treatment_col].values
    ghat=predict_g(df.drop([response_col, treatment_col]+covariate_col, axis=1).values, t, do_SMOTE)
#     ahat_numerator=np.var(g)-np.var(ghat)
#     ahat_denominator=np.mean(ghat*(1-ghat))
    ahat=1-(np.mean(g*(1-g))/np.mean(ghat*(1-ghat)))
    #ahat=ahat_numerator/ahat_denominator
    if ahat<0:
        ahat=0
    return ahat

In [12]:
def plot_sensitivity_graph(df, treatment_col, response_col, covariate_cols, covariate_groups, bias, do_SMOTE):
    #Calculate g and Q
    g=predict_g(df.drop([response_col, treatment_col], axis=1).values, df[treatment_col].values, do_SMOTE, logs=True)
    Q=predict_Q(df.drop([response_col], axis=1).values, df[response_col].values, logs=True)
    calculated_params=pd.DataFrame({'g': g, 'Q': Q})
    
    #Calculate alpha, delta, Rsq
    alpha_series=pd.Series(np.arange(0.0001,1,0.0001))
    #alpha_series=pd.Series(np.arange(0.001,1,0.001))
    delta_series=alpha_series.apply(calc_delta, g=calculated_params['g'], bias=bias)
    sensitivity_df=pd.DataFrame({'alphas':alpha_series, 'deltas':delta_series})
    Rsq=[]
    for n in range(len(alpha_series)):
        Rsq.append(calc_Rsq(alpha_series[n], delta_series[n], calculated_params, df[treatment_col], df[response_col]))
    sensitivity_df['Rsq']=Rsq
    
    #Plot observed co-variates
    alpha_hat=[]
    Rsq_hat=[]
    
    for covar in covariate_cols:
        alpha_hat.append(calc_alpha_hat(covar, response_col, treatment_col, df, calculated_params['g'], do_SMOTE))
        Rsq_hat.append(calc_Rsq_hat(covar, response_col, df, calculated_params['Q']))
    
    variable_importances=pd.DataFrame({'covariates':covariate_cols, 'alpha_hat': alpha_hat, 'Rsq_hat':Rsq_hat, 'groups':covariate_groups})
    
    variable_importances.to_csv('temp_mlflow_files/variable_importances.csv')
    mlflow.log_artifact('temp_mlflow_files/variable_importances.csv')
    treatment_Rsq=calc_Rsq_hat(['treatment'], response_col, df, calculated_params['Q'])
    mlflow.log_metric('treatment_Rsq', treatment_Rsq)
    
    #print(variable_importances)
    
    #print(sensitivity_df)
    
    g=(ggplot(data=sensitivity_df, mapping=aes(x='alphas', y='Rsq'))
            +geom_line(color='#585858', size=1)
            +theme_light()
            +geom_point(data=variable_importances, mapping=aes(x='alpha_hat', y='Rsq_hat', fill='groups'), color="black", alpha=0.6, size=2.5)
            +theme(figure_size=(3.5,3.5), legend_key = element_blank(), axis_title=element_text(size=10), axis_text=element_text(color='black'), plot_title=element_text(size=10))
            +scale_x_continuous(expand=[0, 0, 0, 0], limits=(-0.01,0.43))
            #+scale_y_continuous(expand=[0, 0, 0, 0], limits=(-0.02, min(0.75, max(Rsq+Rsq_hat))))
            +scale_y_continuous(expand=[0, 0, 0, 0], limits=(-0.02, 0.6))
            +labs(x=r'$\mathregular{\alpha}$ (treatment)', fill='', y='partial $R^2$ (outcome)')
            +scale_fill_brewer(type='qual', palette='Set1', labels=['Individual covariates', 'Pre-program earnings', 'Recent earnings'])
            #+geom_label(data=variable_importances, mapping=aes(x='alpha_hat', y='Rsq_hat', label='labels'), size=8, nudge_y=0.03, adjust_text={'expand_text': (2, 2), 'expand_points': (2,2), 'force_text': (0.1, 0.25), 'force_points':(0.2, 0.5)})
            +annotate("text", x=0.35, y=0.065,label='Bias = $'+str(bias), size=8.5, color='#303030')
           )
    g.save('temp_mlflow_files/plot.png')
    mlflow.log_artifact('temp_mlflow_files/plot.png')
    return None
    
    
    
    
    

## Prepare data

In [13]:
col_names=['treatment', 'age', 'education', 'black', 'hispanic', 'married', 'nodegree', 'RE74', 'RE75', 'RE78']

In [14]:
control=pd.read_csv(pathlib.Path.cwd().parents[1].joinpath('data', 'nswre74_control.txt'), header=None, sep=r"\s\s", names=col_names, engine='python')
treatment=pd.read_csv(pathlib.Path.cwd().parents[1].joinpath('data', 'nswre74_treated.txt'), header=None, sep=r"\s\s", names=col_names, engine='python')
psid=pd.read_csv(pathlib.Path.cwd().parents[1].joinpath('data', 'psid_controls.txt'), header=None, sep=r"\s\s", names=col_names, engine='python')

In [15]:
print(control.shape, treatment.shape, psid.shape)

(260, 10) (185, 10) (2490, 10)


In [16]:
imbens1=pd.concat([control,treatment]).reset_index(drop=True)
imbens1.head()

,treatment,age,education,black,hispanic,married,nodegree,RE74,RE75,RE78
0,0.0,23.0,10.0,1.0,0.0,0.0,1.0,0.0,0.0,0.00
1,0.0,26.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,12383.68
2,0.0,22.0,9.0,1.0,0.0,0.0,1.0,0.0,0.0,0.00
3,0.0,18.0,9.0,1.0,0.0,0.0,1.0,0.0,0.0,10740.08
4,0.0,45.0,11.0,1.0,0.0,0.0,1.0,0.0,0.0,11796.47


In [17]:
imbens1=make_data_imbens(imbens1)
imbens1.head()

,treatment,age,education,black,hispanic,married,RE74,RE75,RE78,pos74,pos75
0,0.0,23.0,10.0,1.0,0.0,0.0,0.0,0.0,0.00,0,0
1,0.0,26.0,12.0,0.0,0.0,0.0,0.0,0.0,12383.68,0,0
2,0.0,22.0,9.0,1.0,0.0,0.0,0.0,0.0,0.00,0,0
3,0.0,18.0,9.0,1.0,0.0,0.0,0.0,0.0,10740.08,0,0
4,0.0,45.0,11.0,1.0,0.0,0.0,0.0,0.0,11796.47,0,0


In [18]:
imbens2=pd.concat([treatment, psid]).reset_index(drop=True)

In [19]:
imbens2=make_data_imbens(imbens2)
imbens2.head()

,treatment,age,education,black,hispanic,married,RE74,RE75,RE78,pos74,pos75
0,1.0,37.0,11.0,1.0,0.0,1.0,0.0,0.0,9930.0460,0,0
1,1.0,22.0,9.0,0.0,1.0,0.0,0.0,0.0,3595.8940,0,0
2,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.0,24909.4500,0,0
3,1.0,27.0,11.0,1.0,0.0,0.0,0.0,0.0,7506.1460,0,0
4,1.0,33.0,8.0,1.0,0.0,0.0,0.0,0.0,289.7899,0,0


In [20]:
imbens3=pd.concat([treatment, psid]).reset_index(drop=True)
imbens3['change_earnings']=imbens3['RE78']-imbens3['RE74']
imbens3=make_data_imbens(imbens3)
imbens3=imbens3.drop('RE78', axis=1)
imbens3.head()

,treatment,age,education,black,hispanic,married,RE74,RE75,change_earnings,pos74,pos75
0,1.0,37.0,11.0,1.0,0.0,1.0,0.0,0.0,9930.0460,0,0
1,1.0,22.0,9.0,0.0,1.0,0.0,0.0,0.0,3595.8940,0,0
2,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.0,24909.4500,0,0
3,1.0,27.0,11.0,1.0,0.0,0.0,0.0,0.0,7506.1460,0,0
4,1.0,33.0,8.0,1.0,0.0,0.0,0.0,0.0,289.7899,0,0


In [21]:
imbens4=pd.concat([treatment, psid]).reset_index(drop=True)
imbens4=imbens4[~((imbens4['RE74']>=5000) | (imbens4['RE75']>=5000))]
imbens4=imbens4.reset_index(drop=True)
#Does this match imbens' number of datapoints?
imbens4.groupby('treatment').count()
#yes!

,age,education,black,hispanic,married,nodegree,RE74,RE75,RE78
treatment,,,,,,,,,
0.0,242,242,242,242,242,242,242,242,242
1.0,148,148,148,148,148,148,148,148,148


In [22]:
imbens4=make_data_imbens(imbens4)
imbens4.head()

,treatment,age,education,black,hispanic,married,RE74,RE75,RE78,pos74,pos75
0,1.0,37.0,11.0,1.0,0.0,1.0,0.0,0.0,9930.0460,0,0
1,1.0,22.0,9.0,0.0,1.0,0.0,0.0,0.0,3595.8940,0,0
2,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.0,24909.4500,0,0
3,1.0,27.0,11.0,1.0,0.0,0.0,0.0,0.0,7506.1460,0,0
4,1.0,33.0,8.0,1.0,0.0,0.0,0.0,0.0,289.7899,0,0


In [23]:
covariate_cols=['age', 'education', 'black', 'hispanic', 'married', 'RE74', 'RE75', 'pos74', 'pos75']
recent_earnings=['RE75', 'pos75']
#what about "pre-program earnings"? Is that RE74?
pre_program_earnings=['RE74', 'RE75', 'pos74', 'pos75']

covariates_for_graphs=[[x] for x in covariate_cols]
covariates_for_graphs.append(recent_earnings)
covariates_for_graphs.append(pre_program_earnings)

covariate_groups=['individual_covariates']*len(covariate_cols)+['recent_earnings', 'pre_program_earnings']


## Results

In [26]:
N_EST=100
MAX_DEPTH=5
BIAS=1000

GIT_PYTHON_REFRESH=0
mlflow.start_run()
mlflow.log_param('data', 'imbens2')
mlflow.log_param('bias', BIAS)
mlflow.log_param('model', 'rf')
plot_sensitivity_graph(imbens2, 
                       treatment_col='treatment', 
                       response_col='RE78', 
                       covariate_cols=covariates_for_graphs, 
                       covariate_groups=covariate_groups, 
                       bias=BIAS, 
                       do_SMOTE=False)

mlflow.end_run()

C:\Users\light\Anaconda3\lib\site-packages\plotnine\ggplot.py:729: PlotnineWarning: Saving 3.5 x 3.5 in image.
  from_inches(height, units), units), PlotnineWarning)
C:\Users\light\Anaconda3\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: temp_mlflow_files/plot.png
  warn('Filename: {}'.format(filename), PlotnineWarning)
C:\Users\light\Anaconda3\lib\site-packages\plotnine\geoms\geom_path.py:75: PlotnineWarning: geom_path: Removed 5699 rows containing missing values.
  warn(msg.format(n1-n2), PlotnineWarning)


In [31]:
for data,name in [(imbens1, 'imbens1'), (imbens2, 'imbens2')]:
    for smote in [True, False]:
        for N_EST, MAX_DEPTH in [(100,5), (10,3)]:
            N_EST=N_EST
            MAX_DEPTH=MAX_DEPTH
            BIAS=1000
            GIT_PYTHON_REFRESH=0
            mlflow.start_run()
            mlflow.log_param('data', name)
            mlflow.log_param('bias', BIAS)
            mlflow.log_param('model', 'rf')
            plot_sensitivity_graph(data, 
                                   treatment_col='treatment', 
                                   response_col='RE78', 
                                   covariate_cols=covariates_for_graphs, 
                                   covariate_groups=covariate_groups, 
                                   bias=BIAS, 
                                   do_SMOTE=smote)

            mlflow.end_run()

C:\Users\light\Anaconda3\lib\site-packages\plotnine\ggplot.py:729: PlotnineWarning: Saving 3.5 x 3.5 in image.
  from_inches(height, units), units), PlotnineWarning)
C:\Users\light\Anaconda3\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: temp_mlflow_files/plot.png
  warn('Filename: {}'.format(filename), PlotnineWarning)
C:\Users\light\Anaconda3\lib\site-packages\plotnine\geoms\geom_path.py:75: PlotnineWarning: geom_path: Removed 5784 rows containing missing values.
  warn(msg.format(n1-n2), PlotnineWarning)
C:\Users\light\Anaconda3\lib\site-packages\plotnine\ggplot.py:729: PlotnineWarning: Saving 3.5 x 3.5 in image.
  from_inches(height, units), units), PlotnineWarning)
C:\Users\light\Anaconda3\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: temp_mlflow_files/plot.png
  warn('Filename: {}'.format(filename), PlotnineWarning)
C:\Users\light\Anaconda3\lib\site-packages\plotnine\geoms\geom_path.py:75: PlotnineWarning: geom_path: Removed 5787 rows 